# Proyecto redes neuronales: Uso de autoencoders para la detección de anomalías en tráfico web

## Carga del dataset




In [19]:
from google.colab import drive
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import EarlyStopping

import re

drive.mount('/content/drive')

PROJECT_ROOT = "/content/drive/MyDrive/COLAB_ARCHIVOS/RN_Proyecto"
os.makedirs(PROJECT_ROOT, exist_ok=True)

print("Directorio de trabajo:", PROJECT_ROOT)
print("Contenido actual:", os.listdir(PROJECT_ROOT))

TSF_NAME = "kaggle_web_traffic_dataset.tsf"
TSF_PATH = os.path.join(PROJECT_ROOT, TSF_NAME)

if not os.path.exists(TSF_PATH):
    raise FileNotFoundError(f"No se encontró el archivo TSF en: {TSF_PATH}")

print(f"\nArchivo TSF encontrado correctamente: {TSF_PATH}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Directorio de trabajo: /content/drive/MyDrive/COLAB_ARCHIVOS/RN_Proyecto
Contenido actual: ['kaggle_web_traffic_dataset.tsf', 'Salidas', 'Salidas_multi']

Archivo TSF encontrado correctamente: /content/drive/MyDrive/COLAB_ARCHIVOS/RN_Proyecto/kaggle_web_traffic_dataset.tsf


## Lectura TSF


In [20]:
def load_tsf_dataset(tsf_path):
    with open(tsf_path, "r") as f:
        lines = f.readlines()

    data_start = None
    for i, line in enumerate(lines):
        if line.strip().lower() == "@data":
            data_start = i + 1
            break

    if data_start is None:
        raise ValueError("No se encontró la sección @data en el TSF.")

    series = []
    for line in lines[data_start:]:
        if not line.strip():
            continue
        parts = line.split(",", 1)
        name = parts[0].strip()
        values_str = parts[1].strip()
        values_str = re.sub(r"[\[\]]", "", values_str)
        values = np.array([float(v) if v != "NaN" else np.nan
                           for v in values_str.split(",")])
        series.append((name, values))

    return series

series_list = load_tsf_dataset(TSF_PATH)
print(f"Total de series cargadas: {len(series_list)}")
print("Ejemplo primera serie:", series_list[0][0], "longitud:", len(series_list[0][1]))


Total de series cargadas: 145063
Ejemplo primera serie: T1:2015-07-01 00-00-00:18 longitud: 802


## Carga de TSF

In [21]:
N_SERIES = 10
selected_indices = list(range(min(N_SERIES, len(series_list))))
print("Índices seleccionados:", selected_indices)


all_values_for_scaler = []

for idx in selected_indices:
    name, values = series_list[idx]
    serie = pd.Series(values).fillna(0)
    all_values_for_scaler.append(serie.values.reshape(-1, 1))

all_concat = np.vstack(all_values_for_scaler)
scaler = MinMaxScaler()
scaler.fit(all_concat)

print("Scaler ajustado con todas las series seleccionadas.")


Índices seleccionados: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
Scaler ajustado con todas las series seleccionadas.


## Ventanas globales

In [22]:
def crear_ventanas(data, ventana=30):
    X, y = [], []
    for i in range(len(data) - ventana):
        X.append(data[i:i+ventana])
        y.append(data[i+ventana])
    return np.array(X), np.array(y)

VENTANA = 30

X_global = []
y_global = []
meta_info = []

for s_idx in selected_indices:
    name, values = series_list[s_idx]
    serie = pd.Series(values).fillna(0)
    serie_scaled = scaler.transform(serie.values.reshape(-1, 1))

    X_s, y_s = crear_ventanas(serie_scaled, VENTANA)


    for i in range(len(X_s)):
        t_obj = i + VENTANA
        meta_info.append((s_idx, t_obj))

    X_global.append(X_s)
    y_global.append(y_s)

X_global = np.vstack(X_global)
y_global = np.vstack(y_global)


X_global = X_global.reshape((X_global.shape[0], X_global.shape[1], 1))

print("\nShapes globales:")
print("X_global:", X_global.shape)
print("y_global:", y_global.shape)
print("Tamaño meta_info:", len(meta_info))


Shapes globales:
X_global: (7720, 30, 1)
y_global: (7720, 1)
Tamaño meta_info: 7720


## Entrenamiento de autoencoder


In [23]:
SALIDAS_DIR = os.path.join(PROJECT_ROOT, "Salidas_multi")
os.makedirs(SALIDAS_DIR, exist_ok=True)

model = Sequential([
    LSTM(128, return_sequences=True, input_shape=(VENTANA, 1)),
    LSTM(64, return_sequences=False),
    Dense(64, activation="relu"),
    Dense(1)
])

model.compile(optimizer="adam", loss="mse")
model.summary()

early_stop = EarlyStopping(
    monitor="val_loss",
    patience=5,
    restore_best_weights=True
)

history = model.fit(
    X_global, y_global,
    epochs=50,
    batch_size=64,
    validation_split=0.1,
    callbacks=[early_stop],
    verbose=1
)

history_df = pd.DataFrame(history.history)
history_path = os.path.join(SALIDAS_DIR, "historial_entrenamiento_global.csv")
history_df.to_csv(history_path, index=False)
print("\nHistorial de entrenamiento global guardado en:", history_path)


/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_4 (LSTM)                   │ (None, 30, 128)        │        66,560 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_5 (LSTM)                   │ (None, 64)             │        49,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 120,193 (469.50 KB)

 Trainable params: 120,193 (469.50 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
109/109 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - loss: 0.0011 - val_loss: 1.8752e-04
Epoch 2/50
109/109 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - loss: 0.0012 - val_loss: 1.1160e-04
Epoch 3/50
109/109 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 9.5898e-04 - val_loss: 1.4725e-04
Epoch 4/50
109/109 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 0.0010 - val_loss: 1.1106e-04
Epoch 5/50
109/109 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 9.7068e-04 - val_loss: 1.7988e-04
Epoch 6/50
109/109 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 0.0015 - val_loss: 1.2103e-04
Epoch 7/50
109/109 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 9.9685e-04 - val_loss: 1.1145e-04
Epoch 8/50
109/109 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 0.0010 - val_loss: 2.0738e-04
Epoch 9/50
109/109 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 8.6372e-04 - val_loss: 1.2136e-04

Historial de entrenamiento global guardado en: /content/drive/MyDrive/COLAB_ARCHIVOS/RN_Proyecto/Salidas_multi/historial_entrenamiento_global.csv


## Detección de anomalías

In [24]:
pred_global = model.predict(X_global)
errores = np.mean(np.abs(pred_global - y_global), axis=1)

umbral = np.mean(errores) + 2 * np.std(errores)
anom_mask = errores > umbral
anom_indices = np.where(anom_mask)[0]

print("\nUmbral global de anomalía:", umbral)
print("Número total de ventanas anómalas:", len(anom_indices))

# Organizar anomalías por serie
anom_por_serie = {idx: [] for idx in selected_indices}
for k in anom_indices:
    s_idx, t_obj = meta_info[k]
    anom_por_serie[s_idx].append(t_obj)

for s_idx in selected_indices:
    name, values = series_list[s_idx]
    print(f"Serie {s_idx} ({name}): {len(anom_por_serie[s_idx])} puntos anómalos")


242/242 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step

Umbral global de anomalía: 0.06939156186469969
Número total de ventanas anómalas: 178
Serie 0 (T1:2015-07-01 00-00-00:18): 11 puntos anómalos
Serie 1 (T2:2015-07-01 00-00-00:11): 20 puntos anómalos
Serie 2 (T3:2015-07-01 00-00-00:1): 3 puntos anómalos
Serie 3 (T4:2015-07-01 00-00-00:35): 9 puntos anómalos
Serie 4 (T5:2015-07-01 00-00-00:0): 10 puntos anómalos
Serie 5 (T6:2015-07-01 00-00-00:12): 9 puntos anómalos
Serie 6 (T7:2015-07-01 00-00-00:0): 7 puntos anómalos
Serie 7 (T8:2015-07-01 00-00-00:118): 73 puntos anómalos
Serie 8 (T9:2015-07-01 00-00-00:5): 35 puntos anómalos
Serie 9 (T10:2015-07-01 00-00-00:6): 1 puntos anómalos


## Gráficas


In [25]:
for plot_s_idx in selected_indices:
    page_name, vals = series_list[plot_s_idx]
    serie_plot = pd.Series(vals).fillna(0)

    anom_tiempos = anom_por_serie[plot_s_idx]

    plt.figure(figsize=(14, 5))
    plt.plot(serie_plot.values, label="Tráfico real")

    if anom_tiempos:
        plt.scatter(
            anom_tiempos,
            serie_plot.values[anom_tiempos],
            color="red",
            label="Anomalía",
            zorder=3
        )

    plt.title(f"Detección de anomalías (modelo global)\nSerie {plot_s_idx}: {page_name}")
    plt.xlabel("Tiempo (índice)")
    plt.ylabel("Visitas")
    plt.legend()
    plt.tight_layout()

    anom_fig_path = os.path.join(SALIDAS_DIR, f"anomalias_serie_{plot_s_idx}.png")
    plt.savefig(anom_fig_path)
    plt.close()

    print(f"Gráfico de anomalías guardado para serie {plot_s_idx} en:", anom_fig_path)


Gráfico de anomalías guardado para serie 0 en: /content/drive/MyDrive/COLAB_ARCHIVOS/RN_Proyecto/Salidas_multi/anomalias_serie_0.png
Gráfico de anomalías guardado para serie 1 en: /content/drive/MyDrive/COLAB_ARCHIVOS/RN_Proyecto/Salidas_multi/anomalias_serie_1.png
Gráfico de anomalías guardado para serie 2 en: /content/drive/MyDrive/COLAB_ARCHIVOS/RN_Proyecto/Salidas_multi/anomalias_serie_2.png
Gráfico de anomalías guardado para serie 3 en: /content/drive/MyDrive/COLAB_ARCHIVOS/RN_Proyecto/Salidas_multi/anomalias_serie_3.png
Gráfico de anomalías guardado para serie 4 en: /content/drive/MyDrive/COLAB_ARCHIVOS/RN_Proyecto/Salidas_multi/anomalias_serie_4.png
Gráfico de anomalías guardado para serie 5 en: /content/drive/MyDrive/COLAB_ARCHIVOS/RN_Proyecto/Salidas_multi/anomalias_serie_5.png
Gráfico de anomalías guardado para serie 6 en: /content/drive/MyDrive/COLAB_ARCHIVOS/RN_Proyecto/Salidas_multi/anomalias_serie_6.png
Gráfico de anomalías guardado para serie 7 en: /content/drive/MyDrive